In [56]:
import pyodbc
import pandas as pd
import numpy as np

In [57]:
country_df = pd.read_csv('count_data.csv')
wiki_df = pd.read_csv('wiki_data.csv', thousands=',')

In [58]:
wiki_df.head()

,rank,country,bags,tons,pounds
0,1,Brazil,44200000,2652000,5714381000
1,2,Vietnam,27500000,1650000,3637627000
2,3,Colombia,13500000,810000,1785744000
3,4,Indonesia,11000000,660000,1455050000
4,5,Honduras,9600000,580000,1278681000


In [59]:
wiki_df.columns

Index(['rank', 'country', 'bags', 'tons', 'pounds'], dtype='object')

In [60]:
conn = pyodbc.connect(
    Trusted_Connection="Yes",
    Driver="{ODBC Driver 17 for SQL Server}",
    Server="LAPTOP-3ELSR0G0\SQLEXPRESS",
    Database="Coffee"
)

cursor = conn.cursor()

In [ ]:
cursor.execute('''
               CREATE TABLE Coffee_Data(
                rank int, 
                country nvarchar(50) primary key, 
                bags int, 
                tons int, 
                pounds bigint
               )
               ''')
conn.commit()

In [61]:
print(wiki_df.dtypes)

rank        int64
country    object
bags        int64
tons        int64
pounds      int64
dtype: object


In [62]:
columns_to_convert = ['bags', 'tons', 'pounds']

def remove_quotes(value):

    return str(value).strip('"') if pd.notna(value) else value

for col in columns_to_convert:

    wiki_df[col] = wiki_df[col].apply(remove_quotes)
    
    wiki_df[col] = pd.to_numeric(wiki_df[col], errors='coerce').astype('Int64')

print(wiki_df.dtypes)

wiki_df.to_csv('wiki_data.csv', index=False)

rank        int64
country    object
bags        Int64
tons        Int64
pounds      Int64
dtype: object


In [64]:
for row in wiki_df.itertuples():
    cursor.execute(
        '''INSERT INTO Coffee.dbo.Coffee_Data(rank, country, bags, tons, pounds)
           VALUES (?,?,?,?,?)''',
        row.rank,
        row.country,
        int(row.bags) if pd.notna(row.bags) else None,
        int(row.tons) if pd.notna(row.tons) else None,
        int(row.pounds) if pd.notna(row.pounds) else None
    )

conn.commit()

In [ ]:
cursor.execute("SELECT * FROM Coffee.dbo.Coffee_Data")

rows = cursor.fetchall()

for row in rows:
    print(row)b

In [77]:
count_df = pd.read_csv('count_data.csv')


In [78]:
count_df.columns

Index(['country_name', 'country_capital', 'country_area',
       'country_population'],
      dtype='object')

In [90]:
cursor.execute('''
               CREATE TABLE Country_Data(
                country_name nvarchar(50) primary key, 
                country_capital nvarchar(50), 
                country_area float, 
                country_population int, 
               )
               ''')
conn.commit()

In [79]:
count_df.dtypes

country_name           object
country_capital        object
country_area          float64
country_population      int64
dtype: object

In [85]:
count_df.loc[count_df['country_area'] == float('-inf'), "country_area"] = -1
count_df.loc[count_df["country_area"] == float('inf'), "country_area"] = 1

In [91]:
for row in count_df.itertuples():
    try:
        country_capital = row.country_capital if pd.notna(row.country_capital) else ''

        cursor.execute(
            '''INSERT INTO Coffee.dbo.Country_Data(country_name, country_capital, country_area, country_population)
               VALUES (?,?,?,?)''',
            row.country_name,
            country_capital,
            row.country_area,
            row.country_population
        )
    except ValueError as ve:

        if 'E' in row.country_area:

            scientific_notation_value = float(row.country_area)
            
            cursor.execute(
                '''UPDATE Coffee.dbo.Country_Data SET country_area = ? WHERE country_name = ?''',
                scientific_notation_value,
                row.country_name
            )
        else:
            print(f"Error inserting data for {row.country_name}: {ve}")
    except Exception as e:
        print(f"Error inserting data for {row.country_name}: {e}")

conn.commit()